In [2]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

trainData - (6400, 15, 251, 2)
testData - (1600, 15, 251, 2)


In [4]:
# our dataset - (12800, 251, 2)

def apply_pca(data, n_components=n_comp):
    # data: (cirs, 251, 2) -> 

    reshaped_data = data.reshape(-1, 251)  # (cirs * 2, 251) -> (12800 * 2, 251) -> (25600, 251)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data) # (25600, 251)
    
    pca = PCA(n_components=n_components)
    
    data_pca = pca.fit_transform(data_scaled) # (cirs * 2, n_components) -> (25600, 4)
    data_pca = data_pca.reshape(-1, 2, n_components) # (12800, 2, n_comp)
    data_pca = data_pca.transpose(0, 2, 1)  # (n_samples, n_comp, 2)
    
    return data_pca, scaler, pca

In [5]:
# ----------------- Train -----------------
# Feature Extraction
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  # (6400, 251, 2)
eve_train_CIRs = trainCIR[:, eve_channel, :, :]  # (6400, 251, 2)
train_cirs = np.vstack((alice_train_CIRs, eve_train_CIRs))  # (12800, 251, 2)
print(alice_train_CIRs[0])
#  PCA
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp) # (12800, n_comp, 2)

# Labels
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # '0' for Alice.
eve_train_labels = np.ones(eve_train_CIRs.shape[0])  # '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels)) # (12800,)

train_features = train_cirs_pca.reshape(train_cirs_pca.shape[0], -1) # (12800, n_comp * 2)

[[   -18     34]
 [   -42     59]
 [   113    132]
 [    87    214]
 [   198     73]
 [    20     36]
 [   -15      4]
 [    -3     87]
 [    13     73]
 [   102     95]
 [    53     98]
 [    26    -48]
 [    81    -13]
 [   169      1]
 [   167     69]
 [   181    112]
 [    96     28]
 [    50     38]
 [    80    213]
 [   152    249]
 [   134    124]
 [   196    130]
 [   244     63]
 [    90     12]
 [    72   -190]
 [   208   -177]
 [   348    -26]
 [   420    222]
 [   312    337]
 [    85    268]
 [   -10     79]
 [   103    104]
 [    98    283]
 [   -56    237]
 [   -45    145]
 [     4    240]
 [   213    -86]
 [   333    218]
 [   137    362]
 [  -279    305]
 [  -123     89]
 [    74     72]
 [   136     79]
 [   171    289]
 [   162    454]
 [    58    241]
 [   -23    108]
 [   -31     77]
 [    15     92]
 [    59    234]
 [   101   2607]
 [  -997  11360]
 [ -2819  15324]
 [ -3083  10353]
 [  2186  -4785]
 [  9230  -9419]
 [ 13802  -2065]
 [ 12403   4833]
 [  5800   855

In [52]:
# ----------------- Test -----------------
# Feature Extraction
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((alice_test_CIRs, eve_test_CIRs))  # (3200, 251, 2)

# PCA
reshaped_test_cirs = test_cirs.reshape(-1, 251)
test_cirs_scaled = scaler.transform(reshaped_test_cirs)
test_cirs_pca = pca.transform(test_cirs_scaled)  # (3200 * 2, 4)
# Reshape to original
test_cirs_pca = test_cirs_pca.reshape(-1, 2, n_comp)  # (3200, 2, 4)
test_cirs_pca = test_cirs_pca.transpose(0, 2, 1)  # (3200, 4, 2)

# Labels
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])  # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels)) # (3200,)

test_features = test_cirs_pca.reshape(test_cirs_pca.shape[0], -1) # (3200, 8)

In [53]:
# ----------------- Classification -----------------
# Step 5: Classification using K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_features, train_labels)

predictions = knn.predict(test_features)


In [54]:
# ----------------- Evaluation -----------------
# accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")
# confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# Gamma calculation
gamma = (tp + fn) / (tn + fp)

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 54.41%
tp: 799
tn: 942
fp: 658
fn: 801
MDR: 0.41125
FAR: 0.500625
AR: 0.5440625


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Create a dummy dataset (251 samples, 2 features)
data = np.random.randn(100, 251, 2)

data_pca = []
for cir in data[:2]:
    
    #(251, 2)
    
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir) # Shape: (251, 2)    
    
    # PCA (sample > feature)
    pca = PCA(n_components=1)
    cir_pca_transformed = pca.fit_transform(cir_scaled)  # Shape: (251, n_components)
    print(cir_pca_transformed.shape)
    # data_pca.append(cir_pca_transformed)

# data_pca = np.array(data_pca)  # Shape: (10, 251, 1)
# print(data_pca.shape)


(251, 1)
(251, 1)


In [6]:
import numpy as np

# Load your data
# data = np.loadtxt('prompt_sparse.txt')
data = alice_train_CIRs[0]
print(data.shape)
# Calculate a metric (e.g., product of pairs)
products = np.abs(data[:, 0] * data[:, 1])  # Using absolute value for significance

# Sort by the metric in descending order
top_indices = np.argsort(products)[::-1][:4]
top_components = data[top_indices]

print("Top 4 components based on product:")
print(top_components.shape)

(251, 2)
Top 4 components based on product:
(4, 2)
